In [ ]:
import keras
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os
from pathlib import Path
from tensorflow import keras
import tensorflow as tf


In [ ]:
classes = 5
train = Path(r'C:\Users\Akash Sagar\Downloads\Deep learning\face-expression-recognition-dataset\face-expression-recognition-dataset\train')
valid = Path(r'C:\Users\Akash Sagar\Downloads\Deep learning\face-expression-recognition-dataset\face-expression-recognition-dataset\validation')

# classes = 7
# train = Path(r'C:\Users\Akash Sagar\Downloads\facedetect\dataset\fer2013\train')
# valid = Path(r'C:\Users\Akash Sagar\Downloads\facedetect\dataset\fer2013\validation')

traingen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,rotation_range=30,shear_range=0.3,zoom_range=0.3,width_shift_range=0.4,height_shift_range=0.4,horizontal_flip=True,fill_mode='nearest')
validgen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = traingen.flow_from_directory(train,target_size=(48, 48),batch_size=32,class_mode='categorical',shuffle=True)
validation_generator = validgen.flow_from_directory(valid,target_size=(48, 48),batch_size=32,class_mode='categorical',shuffle=True)
# train_generator = traingen.flow_from_directory(train,color_mode='grayscale',target_size=(48, 48),batch_size=32,class_mode='categorical',shuffle=True)
# validation_generator = validgen.flow_from_directory(valid,color_mode='grayscale',target_size=(48, 48),batch_size=32,class_mode='categorical',shuffle=True)

In [ ]:

model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(keras.layers.Dense(300, activation="relu")) #hidden layers dense
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(keras.layers.Dense(5, activation= "softmax")) # since we have 5 classes 
model.summary()

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(48,48,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Dropout(0.3))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()
from tensorflow.keras.optimizers import RMSprop,SGD,Adam # use  tensorflow.keras.optimizer



In [ ]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam,Nadam # use tensorflow.keras.optimizer
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
checkpoint = ModelCheckpoint("C:/Users/Akash Sagar/Downloads/Deep learning/face-expression-recognition-dataset/modelface1.h5",monitor='val_loss',mode='min',save_best_only=True,verbose=1)

early_stopping_patience = 10
earlystop = EarlyStopping(monitor="accuracy", patience=early_stopping_patience, restore_best_weights=True)   
# earlystop = EarlyStopping(monitor='val_loss',min_delta=0,patience=3,verbose=1,restore_best_weights=True)   

reduce_lr = ReduceLROnPlateau(monitor='accuracy',factor=0.2,patience=3,verbose=1,min_delta=0.0001)      #if no improvemnt seen the it change learning rate

callbacks = [earlystop, checkpoint, reduce_lr]
# callbacks = [checkpoint,earlystop]
model.compile(loss='categorical_crossentropy',optimizer = Nadam(learning_rate=0.001),metrics=['accuracy'])                                            
# model.compile(loss='categorical_crossentropy',optimizer = RMSprop(),metrics=['accuracy'])   


nb_train_samples = 28623
nb_validation_samples = 5937
epoches=50
batchsize =32
history=model.fit(train_generator,steps_per_epoch=nb_train_samples//batchsize,epochs=epoches,callbacks=callbacks,validation_data=validation_generator,validation_steps = nb_validation_samples//batchsize)
# history=model.fit(train_generator,validation_data=validation_generator,epochs=epoches,callbacks=callbacks)



In [ ]:
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
batchsize =32
nb_train_samples = 28623
nb_validation_samples = 5937
# We need to recreate our validation generator with shuffle = false
validation_generator = validgen.flow_from_directory(valid,color_mode = 'grayscale',target_size=(48, 48),batch_size=batchsize,class_mode='categorical',shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
print(class_labels.values())
classes = list(class_labels.values())

print("\n")
# Confution Matrix and Classification Report
Y_pred = model.predict(validation_generator, nb_validation_samples // batchsize+1)
y_pred = np.argmax(Y_pred, axis=1)
print('\tConfusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print("\n")
print('\t \t \tClassification Report')
target_names = list(class_labels.values())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
cnf_matrix = confusion_matrix(validation_generator.classes, y_pred)

plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(len(classes))
_ = plt.xticks(tick_marks, classes, rotation=90)
_ = plt.yticks(tick_marks, classes)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('classic')

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
#face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "C:/Users/Akash Sagar/Downloads/Deep learning/face/haarcascade_frontalface_default.xml")
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
classifier = load_model(r'C:\Users\Akash Sagar\Downloads\Deep learning\face-expression-recognition-dataset\modelface.h5')
class_labels = ['angry', 'happy', 'neutral', 'sad', 'surprise']
# class_labels = ['Angry','Disgust','Fear','Happy', 'Neutral', 'Sad', 'Surprise']
cap = cv2.VideoCapture(0)
while True:

    ret, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:
        #cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)



        if np.sum([roi_gray])!=0:
         roi = roi_gray.astype('float')/255.0
         roi = img_to_array(roi)
         roi = np.expand_dims(roi,axis=0)


            
         preds = classifier.predict(roi)[0]
         label=class_labels[preds.argmax()]
         label_position = (x,y-5)
         cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_DUPLEX,2,(0,255,0),3)
        else:
         cv2.putText(frame,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    cv2.imshow("Emotion Detection",frame)     
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from keras.preprocessing.image import img_to_array

face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img.copy(),cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    allfaces = []   
    rects = []
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation = cv2.INTER_AREA)
        allfaces.append(roi_gray)
        rects.append((x,w,y,h))
    return rects, allfaces, img

img = cv2.imread("C:/Users/Akash Sagar/Downloads/Deep learning/Images/happy.jpg") # Name of File
rects, faces, image = face_detector(img)

i = 0
for face in faces:
    roi = face.astype("float") / 255.0
    roi = img_to_array(roi)
    roi = np.expand_dims(roi, axis=0)

    # make a prediction on the ROI, then lookup the class
    preds = classifier.predict(roi)[0]
    label = class_labels[preds.argmax()]   

    #Overlay our detected emotion on our pic
    label_position = (rects[i][0] + int((rects[i][1]/2)), abs(rects[i][2] - 10))
    i =+ 1
    cv2.putText(image, label, label_position , cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0), 2)
    
cv2.imshow("Emotion Detector", image)
cv2.waitKey(0)

cv2.destroyAllWindows()